# 일상/연애 주제의 한국어 대화 이진 분류 모델
# 참고자료: https://velog.io/@seolini43/%EC%9D%BC%EC%83%81%EC%97%B0%EC%95%A0-%EC%A3%BC%EC%A0%9C%EC%9D%98-%ED%95%9C%EA%B5%AD%EC%96%B4-%EB%8C%80%ED%99%94-BERT%EB%A1%9C-%EC%9D%B4%EC%A7%84-%EB%B6%84%EB%A5%98-%EB%AA%A8%EB%8D%B8-%EB%A7%8C%EB%93%A4%EA%B8%B0%ED%8C%8C%EC%9D%B4%EC%8D%ACColab-%EC%BD%94%EB%93%9C

In [1]:
!pip install transformers

In [2]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [3]:
# GPU 맞게 설정되었는지 확인하느 코드
n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
Tesla T4


In [6]:
import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/NLP/Classification_BERT")
os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/MyDrive/Colab Notebooks/NLP/Classification_BERT'

In [41]:
# 데이터 불러오기
chatbot_data = pd.read_csv('ChatbotData.csv')

# 데이터 정보
## 11,876개의 한글 대화 문답으로 되어 있는 인공데이터로,  일상 대화, 이별과 관련된 대화, 긍정적인 사랑에 대한 대화가 각각 0, 1, 2로 라벨링 되어 있다.
## 컬럼 설명
### Q: 한글 질의
### A: 한글 답변
### label: 일상 대화 - 0, 연애 관련 이별(부정) 대화 - 1, 연애 관련 사랑(긍정) 대화 - 2

In [42]:
chatbot_data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [43]:
chatbot_data.loc[(chatbot_data['label'] == 2), 'label'] = 1

# train-test set 나누기
## 데이터 확인해보면 0~5290까지의 행이 모두 클래스가 0인 행이고, 5291행부터 마지막 행까지 모두 클래스가 1임.
## 1. 특정 비율로 데이터셋을 나눌 때 클래스가 0인 데이터와 1인 데이터 골고루 섞기 위해 데이터 랜덤으로 섞은 뒤 나누는 작업 진행
## 2. 그냥 평상시처럼 train_test_split 함수 사용해서 dataset 나누기

In [44]:
# 1.
chatbot_data_shuffled = chatbot_data.sample(frac=1).reset_index(drop=True)

In [45]:
#train data & test data 로드
train = chatbot_data_shuffled[:9000]
test = chatbot_data_shuffled[9000:]

In [46]:
train['label'].value_counts()

1    4951
0    4049
Name: label, dtype: int64

In [47]:
test['label'].value_counts()

1    1582
0    1241
Name: label, dtype: int64

In [48]:
# 2.
X_train, X_test = train_test_split(chatbot_data, test_size = 0.2 , random_state = 42)
X_train

,Q,A,label
10512,엄청 로맨틱해,생각만해도 달콤하네요.,1
1199,대리님이 너무 갈궈,더 웃으면서 대해보세요.,0
9841,사내커플인데 비밀연애임. 답답해.,비밀연애가 말도 못하고 힘들죠.,1
5595,그 사람이 참 그리워,사랑했나봐요.,1
7228,왜,궁금하네요.,1
...,...,...,...
11284,좋아했지만 고백은 못하겠어.,애틋한 사랑이네요.,1
5191,화장실!!,화장실 가세요.,0
5390,6개월이 지나도 왜이런거죠?,물리적 시간에 비례하지 않으니까요.,1
860,내가 제일 문제인 듯,당신은 하나밖에 없는 소중한 사람이에요.,0


In [49]:
X_train['label'].value_counts()

1    5256
0    4202
Name: label, dtype: int64

In [50]:
X_test['label'].value_counts()

1    1277
0    1088
Name: label, dtype: int64

# Train set 전처리
## 1. BERT 분류 모델의 경우 각 문장의 앞마다 [CLS]를 붙여 인식시키고, 문장의 종료는 [SEP]를 붙여 인식시킨다. [CLS]을 인식함으로써 문장의 처음이라 알 수 있게 하고, [SEP]을 인식함으로써 문장의 끝을 알 수 있게 하기 위함이다.
## 2. BERT에서 사용하는 토크나이저인 WordPiece: 단어를 토큰화할 때, 단어집합에 없는 단어는 더 쪼개서 ##을 붙여주는 방식
## 3. 문장의 최대 시퀀스를 설정하여 정수 인코딩과 제로 패딩을 수행
## 4. Attention Mask: 0 값을 가지는 패딩 토큰에 대해서 어텐션 연산을 불필요하게 수행하지 않도록 단어와 패딩 토큰을 구분할 수 있게 알려주는 것을 말한다. 따라서 [40311, 9435, 102, 0, 0]와 같은 패딩된 데이터가 있을 때, 패딩된 값은 '0', 패딩되지 않은 단어는 '1'의 값을 갖도록 시리얼 데이터를 만들어 주어야 한다.

In [51]:
# 1.
sentences = ["[CLS] " + str(s) + " [SEP]" for s in train.Q]
sentences[:5]

['[CLS] 종이에 손 베였어 [SEP]',
 '[CLS] 일을 거지같이해 [SEP]',
 '[CLS] 7일차 [SEP]',
 '[CLS] 판도라의 상자를 연 기분 [SEP]',
 '[CLS] A형 남자랑 썸을 타게 됨 [SEP]']

In [52]:
# 0과 1 라벨이 담겨 있는 컬럼을 array로 따로 저장
labels = train['label'].values
labels

array([0, 0, 1, ..., 1, 1, 1])

In [53]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", do_lower_case=False)
result = tokenizer.tokenize('안녕하세요')
print(result)
'''현재 단어 집합이 비어있는 상태이므로 '안녕하세요'라는 단어가 단어집합에 없다.
따라서 이 단어는 한번 더 쪼개어 지는데 '##'을 붙임으로써 '##녕', '##하', '##세', '##요' 가 어떠한 단어의 서브워드라는 것을 나타내준다.
이렇게 나타내 줌으로써 단어의 의미를 잃지 않고 다시 원래 단어로 복원을 할 수 있게 된다.'''

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

['안', '##녕', '##하', '##세', '##요']


In [54]:
# 2.
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(s) for s in sentences]

In [55]:
print(sentences[0])  #토크나이징 전
print(tokenized_texts[0]) #토크나이징 후

[CLS] 종이에 손 베였어 [SEP]
['[CLS]', '종', '##이', '##에', '손', '베', '##였', '##어', '[SEP]']


In [57]:
# 3.
MAX_LEN = 128 #최대 시퀀스 길이 설정
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
input_ids[0]

array([   101,   9684,  10739,  10530,   9450,   9344, 119147,  12965,
          102,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [58]:
# 4.
attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
